In [11]:
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import sparse
from scipy.stats.mstats import gmean
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from datetime import datetime
from sklearn.model_selection import train_test_split,StratifiedKFold, KFold
from sklearn.model_selection import cross_val_score
seed = 1234


# Load Data

In [12]:
#input data
train_df=pd.read_json('../input/train.json').reset_index(drop = True)
test_df=pd.read_json('../input/test.json').reset_index(drop = True)

print train_df.shape
print test_df.shape

(49352, 15)
(74659, 14)


In [13]:
data_path = "../input/"
train_X = pd.read_csv(data_path + 'train_BM_MB_add03052240.csv')
test_X  = pd.read_csv(data_path + 'test_BM_MB_add03052240.csv')
train_y = np.ravel(pd.read_csv(data_path + 'labels_BrandenMurray.csv'))
print train_X.shape
print test_X.shape

(49352, 322)
(74659, 322)


In [14]:
# target_num_map = {'high':2, 'medium':1, 'low':0}
# train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

In [15]:
time_feature = pd.read_csv(data_path + 'listing_image_time.csv')
time_feature.columns = ['listing_id','time_stamp']
train_X = train_X.merge(time_feature,on='listing_id',how='left')
test_X = test_X.merge(time_feature,on='listing_id',how='left')

print train_X.shape
print test_X.shape

(49352, 323)
(74659, 323)


In [9]:
def RFC_cv(max_features = 0.5, n_estimators=300, min_samples_leaf  =1):
    scores=[]
    est=RandomForestClassifier(max_features=max_features,
                               n_estimators=int(n_estimators),
                               min_samples_leaf =int(min_samples_leaf), 
                               criterion = 'entropy',
                               random_state=seed,
                               n_jobs = 6
                              )
#     est.fit(X_train, y_train)
#     y_val_pred = est.predict_proba(X_val)
    return cross_val_score(est,train_X,train_y, scoring = 'neg_log_loss', cv = 3).mean()
#     return -1*log_loss(y_val, y_val_pred)

In [10]:
cv_score = -1
for x in [0.1,0.2,0.3,0.4]:
    score = RFC_cv(max_features = x)
    if score > cv_score:
        max_features = x
        cv_score = score
    print x,'\t', score

0.1 	-1.05908693106
0.2 	-1.32274767421


KeyboardInterrupt: 

In [8]:
for x in [0.125,0.15,0.175]:
    score = RFC_cv(max_features = x)
    if score > cv_score:
        max_features = x
        cv_score = score
    print x,'\t', score

0.125 	-0.578582760395
0.15 	-0.578081378388
0.175 	-0.576994845138


In [9]:
print max_features
# 0.1 	-0.569534791531
# 0.2 	-0.567784272359
# 0.3 	-0.57182990891
# 0.4 	-0.577731569868
# 0.5 	-0.576480500868
# 0.6 	-0.578800225854
# 0.7 	-0.576597957953
# 0.8 	-0.579586563
# 0.9 	-0.579104546053
# sqrt 	-0.57092231787
# log2 	-0.5842971373


0.175


In [10]:
for x in [1,2,4,8,16,32]:
    score = RFC_cv(max_features = max_features,min_samples_leaf = x)
    if score > cv_score:
        min_samples_leaf = x
        cv_score = score    
    print x, '\t', score

1 	-0.576994845138
2 	-0.570733465895
4 	-0.569969626993
8 	-0.57126103339
16 	-0.575156616641
32 	-0.580900647303


In [11]:
for x in [3,5,6,7]:
    score = RFC_cv(max_features = max_features,min_samples_leaf = x)
    if score > cv_score:
        min_samples_leaf = x
        cv_score = score    
    print x, '\t', score

3 	-0.571064172588
5 	-0.571200956274
6 	-0.57009644001
7 	-0.57104090788


In [12]:
print min_samples_leaf

4


In [7]:
def RFC_blend(est, train_x, train_y, test_x, fold):
    N_params = len(est)
    print ("Blend %d estimators for %d folds" % (N_params, fold))
    skf = KFold(n_splits=fold,random_state=seed)
    N_class = len(set(train_y))
    
    train_blend_x = np.zeros((train_x.shape[0], N_class*N_params))
    test_blend_x_mean = np.zeros((test_x.shape[0], N_class*N_params))
    test_blend_x_gmean = np.zeros((test_x.shape[0], N_class*N_params))
    scores = np.zeros((fold,N_params))
    best_rounds = np.zeros((fold, N_params))    
    
    for j, ester in enumerate(est):
        print ("Model %d:" %(j+1))
        test_blend_x_j = np.zeros((test_x.shape[0], N_class*fold))

            
        for i, (train_index, val_index) in enumerate(skf.split(train_x)):
            print ("Model %d fold %d" %(j+1,i+1))
            fold_start = time.time() 
            train_x_fold = train_x.iloc[train_index]
            train_y_fold = train_y[train_index]
            val_x_fold = train_x.iloc[val_index]
            val_y_fold = train_y[val_index]            
            

            ester.fit(train_x_fold,train_y_fold)
            
            val_y_predict_fold = ester.predict_proba(val_x_fold)
            score = log_loss(val_y_fold, val_y_predict_fold)
            print ("Score: ", score)
            scores[i,j]=score            
            
            train_blend_x[val_index, (j*N_class):(j+1)*N_class] = val_y_predict_fold
            test_blend_x_j[:,(i*N_class):(i+1)*N_class] = ester.predict_proba(test_x)
            
            print ("Model %d fold %d fitting finished in %0.3fs" % (j+1,i+1, time.time() - fold_start))            

        test_blend_x_mean[:,(j*N_class):(j+1)*N_class] = \
                np.stack([test_blend_x_j[:,range(0,N_class*fold,N_class)].mean(1),
                          test_blend_x_j[:,range(1,N_class*fold,N_class)].mean(1),
                          test_blend_x_j[:,range(2,N_class*fold,N_class)].mean(1)]).T
        
        test_blend_x_gmean[:,(j*N_class):(j+1)*N_class] = \
                np.stack([gmean(test_blend_x_j[:,range(0,N_class*fold,N_class)], axis=1),
                          gmean(test_blend_x_j[:,range(1,N_class*fold,N_class)], axis=1),
                          gmean(test_blend_x_j[:,range(2,N_class*fold,N_class)], axis=1)]).T
            
        print ("Score for model %d is %f" % (j+1,np.mean(scores[:,j])))
    print ("Score for blended models is %f" % (np.mean(scores)))
    return (train_blend_x, test_blend_x_mean, test_blend_x_gmean, scores,best_rounds)

In [17]:
est = [RandomForestClassifier(max_features=0.175,
                              n_estimators=1000,
                              min_samples_leaf = 4,
                              random_state=seed,
                              criterion = 'entropy',
                              n_jobs = -1
                             )]

(train_blend_x_RFC,
 test_blend_x_RFC_mean,
 test_blend_x_RFC_gmean,
 blend_scores_RFC,
 best_rounds_RFC) = RFC_blend(est,
                              train_X,train_y,
                              test_X,
                              5)

Blend 1 estimators for 5 folds
Model 1:
Model 1 fold 1
('Score: ', 0.53577388246219482)
Model 1 fold 1 fitting finished in 126.142s
Model 1 fold 2
('Score: ', 0.53018882571289261)
Model 1 fold 2 fitting finished in 131.074s
Model 1 fold 3
('Score: ', 0.55083665449318053)
Model 1 fold 3 fitting finished in 127.606s
Model 1 fold 4
('Score: ', 0.57290405175628079)
Model 1 fold 4 fitting finished in 129.685s
Model 1 fold 5
('Score: ', 0.57280492177311537)
Model 1 fold 5 fitting finished in 133.081s
Score for model 1 is 0.552502
Score for blended models is 0.552502


In [18]:
test_blend_x_RFC_mean

array([[  2.88217244e-01,   5.32268002e-01,   1.79514754e-01],
       [  9.70890542e-01,   2.58011877e-02,   3.30827006e-03],
       [  9.08911530e-01,   7.85351441e-02,   1.25533264e-02],
       ..., 
       [  9.16499996e-01,   7.22323480e-02,   1.12676562e-02],
       [  9.50234186e-01,   4.91177184e-02,   6.48095238e-04],
       [  6.08851454e-01,   3.52309562e-01,   3.88389837e-02]])

In [19]:
test_blend_x_RFC_gmean

array([[  2.87590748e-01,   5.32113441e-01,   1.79077864e-01],
       [  9.70880218e-01,   2.53642759e-02,   2.78782075e-03],
       [  9.08788373e-01,   7.74552029e-02,   1.23579353e-02],
       ..., 
       [  9.16406213e-01,   7.15029332e-02,   1.07557700e-02],
       [  9.50045402e-01,   4.56787307e-02,   5.42948140e-04],
       [  6.08479725e-01,   3.51918985e-01,   3.84800086e-02]])

In [20]:
now = datetime.now()

name_train_blend = '../blend/train_blend_RFC_entropy_last_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend_mean = '../blend/test_blend_RFC_entropy_mean_last_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend_gmean = '../blend/test_blend_RFC_entropy_gmean_last_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'


print (np.mean(blend_scores_RFC,axis=0))
# print (np.mean(best_rounds_RFC,axis=0))
np.savetxt(name_train_blend,train_blend_x_RFC, delimiter=",")
np.savetxt(name_test_blend_mean,test_blend_x_RFC_mean, delimiter=",")
np.savetxt(name_test_blend_gmean,test_blend_x_RFC_gmean, delimiter=",")

[ 0.55250167]


In [20]:
sub_name = '../output/sub_RFC_entropy_gmean_BM_0322_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'

out_df = pd.DataFrame(test_blend_x_RFC_gmean[:,:3])
out_df.columns = ["low", "medium", "high"]
out_df["listing_id"] = test_X.listing_id.values
out_df.to_csv(sub_name, index=False)

In [21]:
def RFC_cv(max_features = 0.5, n_estimators=300, min_samples_leaf  =1):
    scores=[]
    est=RandomForestClassifier(max_features=max_features,
                               n_estimators=int(n_estimators),
                               min_samples_leaf =int(min_samples_leaf), 
                               criterion = 'gini',
                               random_state=seed,
                               n_jobs = 6
                              )
#     est.fit(X_train, y_train)
#     y_val_pred = est.predict_proba(X_val)
    return cross_val_score(est,train_X,train_y, scoring = 'neg_log_loss', cv = 3).mean()
#     return -1*log_loss(y_val, y_val_pred)

In [22]:
cv_score = -1
for x in [0.1,0.2,0.3,0.4,0.5]:
    score = RFC_cv(max_features = x)
    if score > cv_score:
        max_features = x
        cv_score = score
    print x,'\t', score

0.1 	-0.588520607588
0.2 	-0.585693986069
0.3 	-0.592506736194
0.4 	-0.590559351747
0.5 	-0.595758734446


In [23]:
for x in [0.125,0.15,0.175]:
    score = RFC_cv(max_features = x)
    if score > cv_score:
        max_features = x
        cv_score = score
    print x,'\t', score

0.125 	-0.585233214356
0.15 	-0.586120874595
0.175 	-0.586576410917


In [24]:
print max_features

0.125


In [25]:
for x in [1,2,4,8,16,32]:
    score = RFC_cv(max_features = max_features,min_samples_leaf = x)
    if score > cv_score:
        min_samples_leaf = x
        cv_score = score    
    print x, '\t', score

1 	-0.585233214356
2 	-0.579649905371
4 	-0.576254445688
8 	-0.577511716864
16 	-0.581348649974
32 	-0.587244091747


In [27]:
for x in [3,5,6,7]:
    score = RFC_cv(max_features = max_features,min_samples_leaf = x)
    if score > cv_score:
        min_samples_leaf = x
        cv_score = score    
    print x, '\t', score

3 	-0.576680067161
5 	-0.575859281665
6 	-0.575896736892
7 	-0.577139730886


In [28]:
print min_samples_leaf

5


In [29]:
est = [RandomForestClassifier(max_features=max_features,
                              n_estimators=1000,
                              min_samples_leaf = min_samples_leaf,
                              random_state=seed,
                              criterion = 'gini',
                              n_jobs = 6
                             )]

(train_blend_x_RFC,
 test_blend_x_RFC_mean,
 test_blend_x_RFC_gmean,
 blend_scores_RFC,
 best_rounds_RFC) = RFC_blend(est,
                              train_X,train_y,
                              test_X,
                              10)

Blend 1 estimators for 10 folds
Model 1:
Model 1 fold 1
('Score: ', 0.55121062224642514)
Model 1 fold 1 fitting finished in 87.258s
Model 1 fold 2
('Score: ', 0.53752121146584542)
Model 1 fold 2 fitting finished in 90.931s
Model 1 fold 3
('Score: ', 0.56190327185922007)
Model 1 fold 3 fitting finished in 95.163s
Model 1 fold 4
('Score: ', 0.54314664283549174)
Model 1 fold 4 fitting finished in 93.548s
Model 1 fold 5
('Score: ', 0.56780479126706263)
Model 1 fold 5 fitting finished in 93.776s
Model 1 fold 6
('Score: ', 0.55535427688851013)
Model 1 fold 6 fitting finished in 95.664s
Model 1 fold 7
('Score: ', 0.57019568569245926)
Model 1 fold 7 fitting finished in 93.381s
Model 1 fold 8
('Score: ', 0.58479924361231184)
Model 1 fold 8 fitting finished in 94.669s
Model 1 fold 9
('Score: ', 0.57975761327135855)
Model 1 fold 9 fitting finished in 94.477s
Model 1 fold 10
('Score: ', 0.56528658131160947)
Model 1 fold 10 fitting finished in 93.038s
Score for model 1 is 0.561698
Score for blended

In [30]:
now = datetime.now()

name_train_blend = '../blend/train_blend_RFC_gini_BM_0322_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend_mean = '../blend/test_blend_RFC_gini_mean_BM_0322_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend_gmean = '../blend/test_blend_RFC_gini_gmean_BM_0322_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'


print (np.mean(blend_scores_RFC,axis=0))
# print (np.mean(best_rounds_RFC,axis=0))
np.savetxt(name_train_blend,train_blend_x_RFC, delimiter=",")
np.savetxt(name_test_blend_mean,test_blend_x_RFC_mean, delimiter=",")
np.savetxt(name_test_blend_gmean,test_blend_x_RFC_gmean, delimiter=",")

[ 0.56169799]
